# Create ISA-API Investigation from Datascriptor Study Design configuration
# Crossover Study with radiological and drug treatment on Mice

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
- a 4-arm study design. Each arm has 10 subjects
- subjects are mice. There is an observational factor, named "sex" with two values: "M" and "F"
- a crossover of two treatments, a drug treatment (cyclophosphamide 500 mg for 20 days) and a radiological treatment ("strontium chloride sr 89" 2 MBq/Kg forr 40 days)
- four non-treatment phases: screen, run-in, washout and follow-up
- three sample types colllected: tissue, blood, and urine
- three assay types: 
    - metabolite profiling through mass spectrometry on the urine sample.
    - clinical chemistry with marker on blood samples
    - genomic sequencing on tissue samples using 

## 1. Setup

Let's import all the required libraries

In [1]:
from time import time
import os
import json

In [2]:
## ISA-API related imports
from isatools.model import Investigation, Study

In [3]:
## ISA-API create mode related imports
from isatools.create.model import StudyDesign
from isatools.create.connectors import generate_study_design

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

In [4]:
## ISA-API create mode related imports
from isatools.create import model
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator with all the specs defined above

In [5]:
with open(os.path.abspath(os.path.join(
    "config", "crossover-study-design-2-treatments-mice.json"
)), "r") as config_file:
    study_design_config = json.load(config_file)

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design()` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [6]:
study_design = generate_study_design(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [7]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(identifier='inv01', studies=[study])

The generation of the study design took 1.41 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [8]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 0.60 s.


In [9]:
directory = os.path.abspath(os.path.join('output', 'crossover-2-treatments-mice'))
os.makedirs(directory, exist_ok=True)
with open(os.path.abspath(os.path.join(directory, 'isa-investigation-crossover-2-treatments-mice.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [10]:
start = time()
isatab.dump(investigation, directory)
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The Tab serialisation of the ISA investigation took 29.75 s.


To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [11]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [12]:
len(dataframes)

4

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 2 assay files (one for MS and one for NMR). Let's check the names:

In [13]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_AT8_genome-sequencing_nucleic-acid-sequencing.txt'

'a_AT11_clinical-chemistry_marker-panel.txt'

'a_AT1_metabolite-profiling_mass-spectrometry.txt'

### 7.1 Count of subjects and samples

We have 10 subjects in the each of the 4 arms for a total of 40 subjects.

We collect:
- 7 urine samples per subject (total 280 samples)
- 8 blood samples per subject (total 320 samples)
- 1 tissue sample per subject (total 40 samples)

Across the 4 study arms a total of 640 samples are collected (160 samples per arm)

In [14]:
study_frame = dataframes['s_study_01.txt']
count_arm0_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP0' in el)])
count_arm1_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP1' in el)])
count_arm2_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP2' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP3' in el)])
print("There are {} samples in the GRP0 arm (i.e. group)".format(count_arm0_samples))
print("There are {} samples in the GRP1 arm (i.e. group)".format(count_arm1_samples))
print("There are {} samples in the GRP2 arm (i.e. group)".format(count_arm2_samples))
print("There are {} samples in the GRP3 arm (i.e. group)".format(count_arm3_samples))

There are 160 samples in the GRP0 arm (i.e. group)
There are 160 samples in the GRP1 arm (i.e. group)
There are 160 samples in the GRP2 arm (i.e. group)
There are 160 samples in the GRP3 arm (i.e. group)


### 7.2 Study Table Overview

The study table provides an overview of the subjects (sources) and samples

In [15]:
study_frame

,Source Name,Characteristics[Study Subject],Characteristics[sex],Protocol REF,Parameter Value[Sampling order],Parameter Value[Study cell],Date,Performer,Sample Name,Characteristics[organism part],Comment[study step with treatment],Factor Value[Sequence Order],Factor Value[AGENT],Factor Value[INTENSITY],Unit,Factor Value[DURATION],Unit.1
0,GRP0_SBJ01,Mouse,M,sample collection,157,A0E5,2021-04-16,Unknown,GRP0_SBJ01_A0E5_SMP-Blood-Sample-3,Blood Sample,NO,5,,,,90,days
1,GRP0_SBJ01,Mouse,M,sample collection,155,A0E5,2021-04-16,Unknown,GRP0_SBJ01_A0E5_SMP-Blood-Sample-1,Blood Sample,NO,5,,,,90,days
2,GRP0_SBJ01,Mouse,M,sample collection,069,A0E2,2021-04-16,Unknown,GRP0_SBJ01_A0E2_SMP-Urine-1,Urine,YES,2,strontium chloride sr 89,2.0,MBq/Kg,40,days
3,GRP0_SBJ01,Mouse,M,sample collection,128,A0E5,2021-04-16,Unknown,GRP0_SBJ01_A0E5_SMP-Urine-2,Urine,NO,5,,,,90,days
4,GRP0_SBJ01,Mouse,M,sample collection,058,A0E2,2021-04-16,Unknown,GRP0_SBJ01_A0E2_SMP-Blood-Sample-2,Blood Sample,YES,2,strontium chloride sr 89,2.0,MBq/Kg,40,days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,GRP3_SBJ10,Mouse,F,sample collection,620,A3E5,2021-04-16,Unknown,GRP3_SBJ10_A3E5_SMP-Blood-Sample-1,Blood Sample,NO,5,,,,90,days
636,GRP3_SBJ10,Mouse,F,sample collection,504,A3E0,2021-04-16,Unknown,GRP3_SBJ10_A3E0_SMP-Blood-Sample-1,Blood Sample,NO,0,,,,10,days
637,GRP3_SBJ10,Mouse,F,sample collection,544,A3E2,2021-04-16,Unknown,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,Tissue Sample,YES,2,cyclophosphamide,500.0,mg,20,days
638,GRP3_SBJ10,Mouse,F,sample collection,488,A3E0,2021-04-16,Unknown,GRP3_SBJ10_A3E0_SMP-Urine-2,Urine,NO,0,,,,10,days


### 7.3 First Assay: Metabolite Profiling using mass spectrometry

This assay takes urine samples as input

In [16]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt']

,Sample Name,Comment[study step with treatment],Protocol REF,Performer,Extract Name,Characteristics[extract type],Term Accession Number,Protocol REF.1,Performer.1,Labeled Extract Name,...,Protocol REF.2,Parameter Value[instrument],Term Accession Number.1,Parameter Value[injection_mode],Term Accession Number.2,Parameter Value[acquisition_mode],Term Accession Number.3,MS Assay Name,Performer.2,Raw Spectral Data File
0,GRP0_SBJ01_A0E0_SMP-Urine-1,NO,extraction,Unknown,AT1-S17-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S17-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S17-mass-spectrometry-Acquisition-R3,Unknown,AT1-S17-raw-spectral-data-file-R3.raw
1,GRP0_SBJ01_A0E0_SMP-Urine-1,NO,extraction,Unknown,AT1-S17-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S17-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S17-mass-spectrometry-Acquisition-R2,Unknown,AT1-S17-raw-spectral-data-file-R2.raw
2,GRP0_SBJ01_A0E0_SMP-Urine-1,NO,extraction,Unknown,AT1-S17-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S17-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S17-mass-spectrometry-Acquisition-R4,Unknown,AT1-S17-raw-spectral-data-file-R4.raw
3,GRP0_SBJ01_A0E0_SMP-Urine-1,NO,extraction,Unknown,AT1-S17-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S17-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S17-mass-spectrometry-Acquisition-R1,Unknown,AT1-S17-raw-spectral-data-file-R1.raw
4,GRP0_SBJ01_A0E0_SMP-Urine-2,NO,extraction,Unknown,AT1-S18-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S18-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S18-mass-spectrometry-Acquisition-R1,Unknown,AT1-S18-raw-spectral-data-file-R1.raw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,GRP3_SBJ10_A3E5_SMP-Urine-1,NO,extraction,Unknown,AT1-S267-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S267-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S267-mass-spectrometry-Acquisition-R2,Unknown,AT1-S267-raw-spectral-data-file-R2.raw
1116,GRP3_SBJ10_A3E5_SMP-Urine-2,NO,extraction,Unknown,AT1-S268-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S268-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S268-mass-spectrometry-Acquisition-R2,Unknown,AT1-S268-raw-spectral-data-file-R2.raw
1117,GRP3_SBJ10_A3E5_SMP-Urine-2,NO,extraction,Unknown,AT1-S268-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S268-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S268-mass-spectrometry-Acquisition-R3,Unknown,AT1-S268-raw-spectral-data-file-R3.raw
1118,GRP3_SBJ10_A3E5_SMP-Urine-2,NO,extraction,Unknown,AT1-S268-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S268-LE-R1,...,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S268-mass-spectrometry-Acquisition-R1,Unknown,AT1-

#### Mass Spec Stats

For this assay we have 280 urine samples. 280 DNA extracts are extracted from the samples. The 280 extracts are subsequently labeled. For each labeled extract, 4 mass.spec analyses are run (using Agilent QTQF 6510, positive acquisition mode, 2 replicates each for LC and FIA injection mode), for a total of 1120 mass. spec. processes and 1120 raw spectral data files

In [17]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt'].nunique(axis=0, dropna=True)

Sample Name                            280
Comment[study step with treatment]       2
Protocol REF                             1
Performer                                1
Extract Name                           280
Characteristics[extract type]            1
Term Accession Number                    1
Protocol REF.1                           1
Performer.1                              1
Labeled Extract Name                   280
Label                                    1
Protocol REF.2                           1
Parameter Value[instrument]              1
Term Accession Number.1                  1
Parameter Value[injection_mode]          2
Term Accession Number.2                  2
Parameter Value[acquisition_mode]        1
Term Accession Number.3                  1
MS Assay Name                         1120
Performer.2                              1
Raw Spectral Data File                1120
dtype: int64

### 7.4 Second Assay: Clinical Chemistry Marker Panel

This assay takes blood samples as input

In [18]:
dataframes['a_AT11_clinical-chemistry_marker-panel.txt']

,Sample Name,Comment[study step with treatment],Protocol REF,Performer,Raw Data File
0,GRP0_SBJ01_A0E0_SMP-Blood-Sample-1,NO,sample preparation,Unknown,AT11-S25-raw_data_file-R1
1,GRP0_SBJ01_A0E0_SMP-Blood-Sample-1,NO,sample preparation,Unknown,AT11-S26-raw_data_file-R1
2,GRP0_SBJ01_A0E0_SMP-Blood-Sample-1,NO,sample preparation,Unknown,AT11-S27-raw_data_file-R1
3,GRP0_SBJ01_A0E2_SMP-Blood-Sample-1,YES,sample preparation,Unknown,AT11-S81-raw_data_file-R1
4,GRP0_SBJ01_A0E2_SMP-Blood-Sample-1,YES,sample preparation,Unknown,AT11-S80-raw_data_file-R1
...,...,...,...,...,...
955,GRP3_SBJ10_A3E5_SMP-Blood-Sample-2,NO,sample preparation,Unknown,AT11-S903-raw_data_file-R1
956,GRP3_SBJ10_A3E5_SMP-Blood-Sample-2,NO,sample preparation,Unknown,AT11-S901-raw_data_file-R1
957,GRP3_SBJ10_A3E5_SMP-Blood-Sample-3,NO,sample preparation,Unknown,AT11-S906-raw_data_file-R1
958,GRP3_SBJ10_A3E5_SMP-Blood-Sample-3,NO,sample preparation,Unknown,AT11-S905-raw_data_file-R1


#### Marker Panel Stats

For this assay we use 320 blood samples. For each sample three chemical marker assays are run, producing a total of 960 sample preparation processes and 960 raw data files

In [19]:
dataframes['a_AT11_clinical-chemistry_marker-panel.txt'].nunique(axis=0, dropna=True)

Sample Name                           320
Comment[study step with treatment]      2
Protocol REF                            1
Performer                               1
Raw Data File                         960
dtype: int64

### 7.5 Third Assay: Genome Sequencing

This assay takes the tissue samples as input

In [20]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt']

,Sample Name,Comment[study step with treatment],Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[instrument],Parameter Value[library_orientation],Parameter Value[library_strategy],Performer.1,Raw Data File
0,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S17-Extract-R1,gDNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S17-raw_data_file-R2.raw
1,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S17-Extract-R1,gDNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S17-raw_data_file-R3.raw
2,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S18-Extract-R1,gDNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S18-raw_data_file-R3.raw
3,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S17-Extract-R2,DNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S17-raw_data_file-R8.raw
4,GRP0_SBJ01_A0E4_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S18-Extract-R1,gDNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S18-raw_data_file-R4.raw
...,...,...,...,...,...,...,...,...,...,...,...,...
635,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S67-Extract-R1,gDNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S67-raw_data_file-R3.raw
636,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S67-Extract-R2,DNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S67-raw_data_file-R5.raw
637,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S67-Extract-R1,gDNA,library_preparation,Illumina NovaSeq 6000,single,WGS,Unknown,AT8-S67-raw_data_file-R2.raw
638,GRP3_SBJ10_A3E2_SMP-Tissue-Sample-1,YES,extraction,Unknown,AT8-S67-Extract-R2,DNA,library_preparation,Illumina NovaSeq 6000,single,AMPLICON,Unknown,AT8-S67-raw_data_file-R8.raw


#### Genome Sequencing Stats

For this assay we use 40 tissue samples. For each sample, four extract are extracted (2 DNA and 2 gDNA replicates). Four each extract, four genomic sequencing assays are run (using Illumina NovaSeq 6000, single-end, 2 replicates for WGS and amplicon each), producing a total of 640 raw data files

In [21]:
dataframes['a_AT8_genome-sequencing_nucleic-acid-sequencing.txt'].nunique(axis=0, dropna=True)

Sample Name                              40
Comment[study step with treatment]        1
Protocol REF                              1
Performer                                 1
Extract Name                            160
Characteristics[extract type]             2
Protocol REF.1                            1
Parameter Value[instrument]               1
Parameter Value[library_orientation]      1
Parameter Value[library_strategy]         2
Performer.1                               1
Raw Data File                           640
dtype: int64